### Step 1 ####

The first step is to extract all the stock information from halalstocks.in

This site includes both BSE and NSE stocks, our focus is on getting only NSE stocks

We will apply further filtering later but for now, save all the NSE stocks from this website to an excel file

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Create a persistent session for halalstocks.in
session_halalstocks = requests.Session()
session_halalstocks.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.5",
})

# URL of the website
url = "https://halalstock.in/halal-shariah-compliant-shares-list/"

# Send an HTTP GET request with headers
response = session_halalstocks.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.text, "html.parser")

    # Find the table containing the data
    table = soup.find("table")

    # Extract column headers
    headers = [th.text.strip() for th in table.select("thead th")]

    # Use pandas to read the HTML table into a DataFrame
    df = pd.read_html(str(table))[0]

    # Rename columns with extracted headers
    df.columns = headers

    # Process the first column based on the 'src' attribute of the image tags
    df['Halal'] = [img['src'] for img in table.select("td.column-1 img")]

    # Filter rows based on the 'Halal' column containing 'hs-yes.jpg'
    filtered_df = df[df['Halal'].str.contains('hs-yes.jpg', na=False)]

    # Exclude both the first and last columns before exporting to Excel
    filtered_df = filtered_df.iloc[:, 1:-1]
    
    filtered_df = filtered_df.drop('BSE-ID', axis=1)
    # Remove rows where 'NSECode' column is empty
    filtered_df = filtered_df.dropna(subset=['NSECode'])
    
    # Sort the rows by the "Industry" column
    sorted_df = filtered_df.sort_values(by="Industry")

    # Export the sorted DataFrame to an Excel file
    #sorted_df.to_excel("halal_shares_list_sorted_by_industry.xlsx", index=False)
    sorted_df.to_csv("halal_shares_list_sorted_by_industry.csv", index=False)

    print("Filtered and sorted data exported to Excel successfully.")
else:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")
    
# Close the session for halalstocks.in
session_halalstocks.close()


### Step 2 ###
Download the list of all NSE stocks from nseindia.com from the following url
https://nsearchives.nseindia.com/content/equities/EQUITY_L.csv

In [ ]:
import requests
from requests import Session
url = "https://nsearchives.nseindia.com/content/equities/EQUITY_L.csv"


s=Session()

# Emulate browser
s.headers.update({"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36"})

# Get the cookies from the main page (will update automatically in headers)
s.get("https://www.nseindia.com/")

response = s.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Save the content to a local file
    with open("EQUITY_L.csv", "wb") as file:
        file.write(response.content)
    print("File downloaded successfully.")
else:
    print(f"Failed to download the file. Status code: {response.status_code}")


### Step 3 ###

Now merge both the files from step 1 and step 2
(i.e) check for halal stocks from nse equity list and save it to another file, keep all the columns from step1

In [ ]:
import pandas as pd

# Read the two sheets
df1 = pd.read_csv("halal_shares_list_sorted_by_industry.csv", delimiter=',')
df2 = pd.read_csv("EQUITY_L.csv", delimiter=',')

# Check if 'NSECode' in df1 is present in 'SYMBOL' column of df2
filtered_df = df1[df1['NSECode'].isin(df2['SYMBOL'])]

# Save the result to halal_list.csv
filtered_df.to_csv("halal_list.csv", index=False, sep=',')


### Step 4 ###

Download the totalTradedVolume, totalMarketCap, deliveryToTradedQuantity, Industry, SubIndustry from NSE website
'https://www.nseindia.com/api/quote-equity?symbol='

In [ ]:
import requests
from requests import Session
import pandas as pd
from IPython.display import clear_output
# Create a new session for NSE data
session_nse = requests.Session()


session_nse.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.5",
})

# Define the NSE data URL
#url = f'https://www.nseindia.com/api/quote-equity?symbol={symbol.replace(" ", "%20").replace("&", "%26")}&section=trade_info'

session_nse.get("https://www.nseindia.com/")

# Function to fetch NSE data
def nsefetch(url):
    try:

        # Fetch NSE data
        response = session_nse.get(url)
        response.raise_for_status()
        data = response.json()

 

        return data
    except requests.exceptions.HTTPError as errh:
        print("HTTP Error:", errh)
    except requests.exceptions.ConnectionError as errc:
        print("Error Connecting:", errc)
    except requests.exceptions.Timeout as errt:
        print("Timeout Error:", errt)
    except requests.exceptions.RequestException as err:
        print("Error:", err)
        print("Response content:", response.text)  # Print the response content for further inspection


#Getting FNO Symboles
def fnolist():
    # df = pd.read_csv("https://www1.nseindia.com/content/fo/fo_mktlots.csv")
    # return [x.strip(' ') for x in df.drop(df.index[3]).iloc[:,1].to_list()]

    positions = nsefetch('https://www.nseindia.com/api/equity-stockIndices?index=SECURITIES%20IN%20F%26O')

    nselist=['NIFTY','NIFTYIT','BANKNIFTY']

    i=0
    for x in range(i, len(positions['data'])):
        nselist=nselist+[positions['data'][x]['symbol']]

    return nselist


def nse_quote(symbol,section=""):
    #https://forum.unofficed.com/t/nsetools-get-quote-is-not-fetching-delivery-data-and-delivery-can-you-include-this-a>    symbol = nsesymbolpurify(symbol)

#     if(section==""):
        #if any(x in symbol for x in fnolist()):
            #payload = nsefetch('https://www.nseindia.com/api/quote-derivative?symbol='+symbol)
        #else:
        payload = nsefetch('https://www.nseindia.com/api/quote-equity?symbol='+symbol)
        return payload

#     if(section!=""):
#         payload = nsefetch('https://www.nseindia.com/api/quote-equity?symbol='+symbol+'&section='+section)
#         return payload
    
# Load the Excel file with NSECode column
excel_file_path = "halal_list.csv"
#df = pd.read_excel(excel_file_path)
df = pd.read_csv(excel_file_path)
totalTradedVolume=0
totalMarketCap=0
deliveryToTradedQuantity=0
sector=None
industry=None
BasicIndustry=None
# Iterate over rows and fetch NSE data
for index, row in df.iterrows():
    symbol = row['NSECode']
    url = f'https://www.nseindia.com/api/quote-equity?symbol={symbol.replace(" ", "%20").replace("&", "%26")}&section=trade_info'
    nsedata = nsefetch(url)

#     # Extract required data from NSE response or set to zero if not present
#     try:
#         totalTradedVolume = nsedata['marketDeptOrderBook']['tradeInfo']['totalTradedVolume']
#     except KeyError:
#         totalTradedVolume = 0

#     try:
#         totalMarketCap = nsedata['marketDeptOrderBook']['tradeInfo']['totalMarketCap']
#     except KeyError:
#         totalMarketCap = 0

#     try:
#         deliveryToTradedQuantity = nsedata['securityWiseDP']['deliveryToTradedQuantity']
#     except KeyError:
#         deliveryToTradedQuantity = 0
    
#     nsedata2 = nse_quote(symbol)
#     #print(nsedata2)
#     try:
#         sector = nsedata2['industryInfo']['sector']
#     except KeyError:
#         sector = None
        
#     try:
#         industry = nsedata2['industryInfo']['industry']
#     except KeyError:
#         industry = None
    
#     try:
#         basicIndustry = nsedata2['industryInfo']['basicIndustry']
#     except KeyError:
#         basicIndustry = None
        
#     print(f"sector: {sector}, Industry: {industry}, subIndustry:{basicIndustry}")
#     print(f"row: {index}, NSECode: {symbol}, Total Traded Volume: {totalTradedVolume}, Total Market Cap: {totalMarketCap}, Delivery to Traded Quantity: {deliveryToTradedQuantity}")

#     # Update the DataFrame with new columns
#     df.at[index, 'TotalTradedVolume'] = totalTradedVolume
#     df.at[index, 'TotalMarketCap'] = totalMarketCap
#     df.at[index, 'DeliveryToTradedQuantity'] = deliveryToTradedQuantity

#     # Update the industry and basicIndustry columns
#     df.at[index, 'sector'] = sector
#     df.at[index, 'industry'] = industry
#     df.at[index, 'basicIndustry'] = basicIndustry

# Check if data is fetched successfully
    if nsedata is not None:
        try:
            # Extract required data from NSE response or set to zero if not present
            totalTradedVolume = nsedata.get('marketDeptOrderBook', {}).get('tradeInfo', {}).get('totalTradedVolume', 0)
            totalMarketCap = nsedata.get('marketDeptOrderBook', {}).get('tradeInfo', {}).get('totalMarketCap', 0)
            deliveryToTradedQuantity = nsedata.get('securityWiseDP', {}).get('deliveryToTradedQuantity', 0)

            # Update the DataFrame with new columns
            df.at[index, 'TotalTradedVolume'] = totalTradedVolume
            df.at[index, 'TotalMarketCap'] = totalMarketCap
            df.at[index, 'DeliveryToTradedQuantity'] = deliveryToTradedQuantity

            # Fetch additional data from NSE quote
            nsedata2 = nse_quote(symbol)
            if nsedata2 is not None:
                # Extract additional data from NSE response
                df.at[index, 'sector'] = nsedata2.get('industryInfo', {}).get('sector')
                df.at[index, 'industry'] = nsedata2.get('industryInfo', {}).get('industry')
                df.at[index, 'basicIndustry'] = nsedata2.get('industryInfo', {}).get('basicIndustry')
            else:
                df.at[index, 'sector'] = None
                df.at[index, 'industry'] = None
                df.at[index, 'basicIndustry'] = None
            
            clear_output(wait=True)
            print(f"Row: {index}, NSECode: {symbol}, Total Traded Volume: {totalTradedVolume}, Total Market Cap: {totalMarketCap}, Delivery to Traded Quantity: {deliveryToTradedQuantity}")
            print(f"sector: {df.at[index, 'sector']}, Industry: {df.at[index, 'industry']}, subIndustry:{df.at[index, 'basicIndustry'] }")
        except Exception as e:
            print(f"Error processing row {index}: {e}")

# Save the updated DataFrame to the same Excel file
#df.to_csv(excel_file_path, index=False)

print("Updated data exported to Excel successfully.")

# Close the session for NSE data
session_nse.close()

In [ ]:
import openpyxl
# df_sorted = df.sort_values(['sector', 'industry', 'basicIndustry']).reset_index(drop=True)
df_sorted = df.sort_values(['sector', 'industry', 'basicIndustry', 'TotalMarketCap'], ascending=[True, True, True, False]).reset_index(drop=True)
# df_sorted.to_csv(excel_file_path, index=False)
df_sorted.to_excel("halal_list.xlsx", index=False)
print("Updated data exported to Excel successfully.")